In [ ]:
import upsetplot as up
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import re

In [ ]:
species = "horse"
bed_file = 'horse/'+'sep1_244_gene_ant.bed' # contains loci for all sep1 coordinates
base_bed_file = 'horse/gene_ant_full.txt'
results_file = 'horse/'+'v2output_sep1.txt'# contains VCF info for every sample that has SV in one of the identified coordinates

In [ ]:
# get all loci
entries = []
with open(bed_file) as f:
    lines = f.readlines()
    for line in lines: #
        line = line.split()
        end = int(line[1]) + int(line[2])
        t = ">"+line[0]+":"+line[1]+"-"+str(line[1])
        entries.append(t)
meta_data = pd.read_csv('horse/horse_sra_simple.csv')
meta_data.columns = ['sra','breed']
color_col = []
for index, row in meta_data.iterrows():
    row['breed'] = row['breed'] + str(random.randint(0,9))
    #row['breed'] = row['breed']
#row['color'] = color_col
    #print(meta_data)

In [ ]:
samples = {}

with open(results_file) as f:
    lines = f.readlines()
    for line in lines:
        if "output" in line: # get sample name for each cluster
            sample_name = line.split('/')[1].strip() 
            breed_name = meta_data.loc[meta_data['sra'] == sample_name]
            #print(breed_name, sample_name)
            breed_name = breed_name['breed'].values[0]
            samples[breed_name] = np.zeros(len(entries)) 
        else: # if not a sample name then get vcf info
            t = line.split()
            chrom = t[0]
            start = t[1]
            end = start
            seq = re.search('SVINSSEQ=(.*);SPLIT_READS',line)
            seq = seq.group(1)
            s = ">"+chrom+":"+start+"-"+end
            if s in entries and chrom not in  ['X', 'Y']: # check which SV in reference to TE loci 
                idx = entries.index(s)
                samples[breed_name][idx] = 1


In [43]:
df=pd.DataFrame.from_dict(samples,orient='index').transpose()
cols = list(df.columns.values)
cols = sorted(cols, key=str.lower)
df_new = df[cols]
df_new = df_new.T
print(df_new.index)

Index(['AKHAL-TEKE3', 'AKHAL-TEKE7', 'AKHAL-TEKE9', 'ARABIAN1', 'ARABIAN3',
       'ARABIAN4', 'ARABIAN5', 'ARABIAN6', 'CRIOLLO0', 'CURLY HORSE2',
       'DEBAO1', 'DEBAO5', 'FRIESIAN0', 'FRIESIAN2', 'FRIESIAN7', 'FRIESIAN9',
       'HANOVARIAN7', 'JEJU HORSE9', 'MONGOLIAN1', 'MONGOLIAN2', 'MONGOLIAN5',
       'MONGOLIAN8', 'PRZEWALSKI1', 'PRZEWALSKI2', 'PRZEWALSKI4',
       'PRZEWALSKI7', 'QUARTER HORSE0', 'QUARTER HORSE1', 'QUARTER HORSE3',
       'QUARTER HORSE6', 'QUARTER HORSE8', 'THOROUGHBRED2', 'THOROUGHBRED4',
       'THOROUGHBRED5', 'THOROUGHBRED8', 'THOROUGHBRED9'],
      dtype='object')


In [47]:
print(samples.keys())
ser = []
for col in list(df_new.columns):
    ser.append(pd.Series(df[col].values))

#cat_series = up.from_contents(samples)
#cat_series = [pd.Series(True, index=list(elements), name=name) for name, elements in samples.items()]

#print(cat_series)
#ser = pd.concat(cat_series)

#up.UpSet(ser).plot()

dict_keys(['HANOVARIAN7', 'PRZEWALSKI4', 'PRZEWALSKI1', 'PRZEWALSKI7', 'PRZEWALSKI2', 'QUARTER HORSE0', 'QUARTER HORSE3', 'THOROUGHBRED4', 'QUARTER HORSE6', 'THOROUGHBRED8', 'THOROUGHBRED5', 'QUARTER HORSE8', 'THOROUGHBRED2', 'ARABIAN4', 'ARABIAN3', 'ARABIAN5', 'MONGOLIAN2', 'MONGOLIAN1', 'MONGOLIAN8', 'MONGOLIAN5', 'AKHAL-TEKE3', 'AKHAL-TEKE9', 'FRIESIAN9', 'FRIESIAN0', 'THOROUGHBRED9', 'JEJU HORSE9', 'CURLY HORSE2', 'FRIESIAN7', 'FRIESIAN2', 'DEBAO1', 'DEBAO5', 'CRIOLLO0', 'AKHAL-TEKE7', 'ARABIAN6', 'ARABIAN1', 'QUARTER HORSE1'])


KeyError: 0

In [ ]:

#df_final = df_new.iloc[0:900]
#df_final = df_final.transpose()